# TensorFlow MNIST チュートリアル

* TensorFlowでMNISTサンプルの学習・推論を行う備忘録。
* 参考元
    * [MNIST For ML Beginners](https://www.tensorflow.org/versions/r0.10/tutorials/mnist/beginners/index.html)
    * [Deep MNIST for Experts](https://www.tensorflow.org/versions/r0.10/tutorials/mnist/pros/index.html#deep-mnist-for-experts)
    * [TensorFlow Mechanics 101](https://www.tensorflow.org/versions/master/tutorials/mnist/tf/index.html#tensorflow-mechanics-101)

### 目次

1. MLPでMNIST
2. CNNでMNIST

## MLPでMNIST
* [MNIST For ML Beginners](https://www.tensorflow.org/versions/r0.10/tutorials/mnist/beginners/index.html)の内容

まずはデータを読み込む。
データは、以下のように分割されている。
* 55000件は、学習用データ(mnist.train)
* 10000件は、試験用データ(mnist.test)
* 5000件は、評価用データ(mnist.validation)
また、画像データをxとし、そのラベルをyとする。

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


モデルを実装する。placeholderはTensorFlowで計算を実行するときに利用する変数である。

In [2]:
import tensorflow as tf

In [3]:
x = tf.placeholder(tf.float32, [None, 784])

### モデルの構築

重み変数とバイアスについても定義する。

In [4]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

するとsoftmax回帰モデルは下記のように定義できる。

In [5]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

### 学習

学習させるためにモデルの良し悪しを測る指標が必要である。その１つとして、交差エントロピーがある。

$H_{y'}(y) = -\sum_i y'_i \log(y_i)$

交差エントロピーを実装するために、新しいplaceholderを定義する。

In [6]:
y_ = tf.placeholder(tf.float32, [None, 10])

次いで交差エントロピーを定義する。

In [7]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),  reduction_indices=[1]))

作成した変数の初期化をする演算を追加する。

In [8]:
init = tf.initialize_all_variables()

学習係数を0.5として、逆伝搬による学習のアルゴリズムを選択する。

In [9]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

セッションを作り、変数を初期化したのち、1000回学習を実施する。

In [10]:
sess = tf.Session()
sess.run(init)
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

* モデルを評価する。
    * 予測したラベルと正解ラベルが正しいか
    * 下記はbooleanのリストが返却されるので、平均を取ってみる。

In [11]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [12]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9208


## CNNでMNIST
* [Deep MNIST for Experts](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html)の内容
* MLPで得られた92%という精度をさらに向上するようなモデル

In [13]:
sess = tf.InteractiveSession()

### モデルの構築

重みとバイアスを多数作るため、以下のようなメソッドを用意する。

In [14]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)
 
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

畳込み層とプーリング層を定義する。これも使いまわせるようにメソッドとして定義しておく。Tensorflowではストライドするフィルターの定義や端の処理(padding)は、以下のように扱うことができる。

In [15]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
 
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

第１畳込み層を定義する。
* 特徴のサイズ5x5
* 32の特徴

shapeのフォーマットは、0, 1次元がパッチサイズ、2次元は入力チャネル数, 最後が出力チャネルの数を表す。

In [16]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

この層を適用するために、入力画像データを4次元テンソルに変換する。2、3次元が画像の幅と高さにあたり、最後は色に当たる(ここではグレースケール)。

In [17]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

以下ではx_imageを畳み込み、その出力結果にmax pool層を適用する。

In [18]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

第２畳込み層を定義する。

In [19]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
 
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

全結合層を定義する。

In [20]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
 
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

過学習を抑える施策として、ドロップアウト(ニューロンの間引き)を行う。
学習のときはDropoutを有効にして、推論の際は無効にする。

In [21]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

読み出し層を定義する。
* softmax層を適用

In [22]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
 
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

### 学習

* 最適化アルゴリズムとして、ADAM最適化を利用

In [23]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.arg_max(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.initialize_all_variables())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
    train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})

print("test accuracy %g" % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.1


TypeError: Fetch argument None of None has invalid type <type 'NoneType'>, must be a string or Tensor. (Can not convert a NoneType into a Tensor or Operation.)

## Cifar-10